### 1. 安装HuggingFace 并下载模型到本地

In [2]:
!pip install huggingface-hub -Uqq
!pip install -U sagemaker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.8/857.8 kB 6.9 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for sagemaker: filename=sagemaker-2.177.1-py2.py3-none-any.whl size=1166001 sha256=4d974d413f1f94e8f7623d5e37e63200ff49f761237cda0fd253383ddf1e6e92
  Stored in directory: /root/.cache/pip/wheels/4d/3f/29/a44088e382c815fd842f7d001b32aede28c9fdba7fd58ecf00
Successfully built sagemaker
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.175.0
    Uninstalling sagemaker-2.175.0:
      Successfully uninstalled sagemaker-2.175.0


-  可以把下面cell中的en全部切换成zh即可部署中文版

In [3]:
from huggingface_hub import snapshot_download
from pathlib import Path

local_model_path = Path("./bge-en-model")
local_model_path.mkdir(exist_ok=True)
model_name = "BAAI/bge-large-en"
commit_hash = "48db88180e8c78ac7def7d8e359f2af4f7728a27"

In [4]:
snapshot_download(repo_id=model_name, revision=commit_hash, cache_dir=local_model_path)

'bge-en-model/models--BAAI--bge-large-en/snapshots/48db88180e8c78ac7def7d8e359f2af4f7728a27'

### 2. 把模型拷贝到S3为后续部署做准备

In [5]:
import sagemaker
from sagemaker import image_uris
import boto3
import os
import time
import json

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts

region = sess._region_name
account_id = sess.account_id()

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

In [7]:
s3_model_prefix = "LLM-RAG/workshop/bge-en-model"  # folder where model checkpoint will go
model_snapshot_path = list(local_model_path.glob("**/snapshots/*"))[0]
s3_code_prefix = "LLM-RAG/workshop/bge_en_deploy_code"
print(f"s3_code_prefix: {s3_code_prefix}")
print(f"model_snapshot_path: {model_snapshot_path}")

s3_code_prefix: LLM-RAG/workshop/bge_en_deploy_code
model_snapshot_path: bge-en-model/models--BAAI--bge-large-en/snapshots/48db88180e8c78ac7def7d8e359f2af4f7728a27


In [8]:
!aws s3 cp --recursive {model_snapshot_path} s3://{bucket}/{s3_model_prefix}

upload: bge-en-model/models--BAAI--bge-large-en/snapshots/48db88180e8c78ac7def7d8e359f2af4f7728a27/config.json to s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/bge-en-model/config.json
upload: bge-en-model/models--BAAI--bge-large-en/snapshots/48db88180e8c78ac7def7d8e359f2af4f7728a27/1_Pooling/config.json to s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/bge-en-model/1_Pooling/config.json
upload: bge-en-model/models--BAAI--bge-large-en/snapshots/48db88180e8c78ac7def7d8e359f2af4f7728a27/modules.json to s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/bge-en-model/modules.json
upload: bge-en-model/models--BAAI--bge-large-en/snapshots/48db88180e8c78ac7def7d8e359f2af4f7728a27/tokenizer_config.json to s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/bge-en-model/tokenizer_config.json
upload: bge-en-model/models--BAAI--bge-large-en/snapshots/48db88180e8c78ac7def7d8e359f2af4f7728a27/.gitattributes to s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/bge-en-mod

### 3. 模型部署准备（entrypoint脚本，容器镜像，服务配置）

In [9]:
inference_image_uri = (
    f"763104351884.dkr.ecr.{region}.amazonaws.com/djl-inference:0.23.0-deepspeed0.9.5-cu118"
)

#中国区需要替换为下面的image_uri
# inference_image_uri = (
#     f"727897471807.dkr.ecr.{region}.amazonaws.com.cn/djl-inference:0.21.0-deepspeed0.8.3-cu117"
# )

print(f"Image going to be used is ---- > {inference_image_uri}")

Image going to be used is ---- > 763104351884.dkr.ecr.us-east-2.amazonaws.com/djl-inference:0.23.0-deepspeed0.9.5-cu118


In [11]:
!pip install sentence-transformers -q

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [13]:
!pip show sentence-transformers

Name: sentence-transformers
Version: 2.2.2
Summary: Multilingual text embeddings
Home-page: https://github.com/UKPLab/sentence-transformers
Author: Nils Reimers
Author-email: info@nils-reimers.de
License: Apache License 2.0
Location: /opt/conda/lib/python3.7/site-packages
Requires: huggingface-hub, nltk, numpy, scikit-learn, scipy, sentencepiece, torch, torchvision, tqdm, transformers
Required-by: 


In [14]:
!mkdir -p bge_en_deploy_code

In [38]:
%%writefile bge_en_deploy_code/model.py
from djl_python import Input, Output
import torch
import logging
import math
import os
# from FlagEmbedding import FlagModel
from sentence_transformers import SentenceTransformer


def load_model(properties):
    tensor_parallel = properties["tensor_parallel_degree"]
    model_location = properties['model_dir']
    if "model_id" in properties:
        model_location = properties['model_id']
    logging.info(f"Loading model in {model_location}")

    # model =  FlagModel(model_location)
    model = SentenceTransformer(model_location)
    
    return model

model = None

def handle(inputs: Input):
    global model
    if not model:
        model = load_model(inputs.get_properties())

    if inputs.is_empty():
        return None
    data = inputs.get_as_json()
    
    input_sentences = None
    inputs = data["inputs"]
    if isinstance(inputs, list):
        input_sentences = inputs
    else:
        input_sentences =  [inputs]
        
    is_query = data["is_query"]
    instruction = data["instruction"]
    logging.info(f"inputs: {input_sentences}")
    logging.info(f"is_query: {is_query}")
    logging.info(f"instruction: {instruction}")
    
    if is_query and instruction:
        input_sentences = [ instruction + sent for sent in input_sentences ]
        
    sentence_embeddings =  model.encode(input_sentences,normalize_embeddings=True)
        
    result = {"sentence_embeddings": sentence_embeddings}
    return Output().add_as_json(result)

Overwriting bge_en_deploy_code/model.py


In [39]:
print(f"option.s3url ==> s3://{bucket}/{s3_model_prefix}/")

option.s3url ==> s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/bge-en-model/


#### Note: option.s3url 需要按照自己的账号进行修改, 可以拷贝上一个cell的输出

In [40]:
%%writefile bge_en_deploy_code/serving.properties
engine=Python
option.tensor_parallel_degree=1
option.s3url = s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/bge-en-model/

Overwriting bge_en_deploy_code/serving.properties


In [41]:
%%writefile bge_en_deploy_code/requirements.txt
transformers==4.30.2
sentence-transformers

Overwriting bge_en_deploy_code/requirements.txt


In [42]:
!rm s2e_model.tar.gz
!cd bge_en_deploy_code && rm -rf ".ipynb_checkpoints"
!tar czvf s2e_model.tar.gz bge_en_deploy_code

bge_en_deploy_code/
bge_en_deploy_code/model.py
bge_en_deploy_code/requirements.txt
bge_en_deploy_code/serving.properties


In [43]:
s3_code_artifact = sess.upload_data("s2e_model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {s3_code_artifact}")

S3 Code or Model tar ball uploaded to --- > s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/bge_en_deploy_code/s2e_model.tar.gz


### 4. 创建模型 & 创建endpoint

In [44]:
from sagemaker.utils import name_from_base
import boto3

model_name = name_from_base("bge-large-en") #Note: Need to specify model_name
print(model_name)
print(f"Image going to be used is ---- > {inference_image_uri}")

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={
        "Image": inference_image_uri,
        "ModelDataUrl": s3_code_artifact
    },
    
)
model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

bge-large-en-2023-08-17-10-15-55-952
Image going to be used is ---- > 763104351884.dkr.ecr.us-east-2.amazonaws.com/djl-inference:0.23.0-deepspeed0.9.5-cu118
Created Model: arn:aws:sagemaker:us-east-2:946277762357:model/bge-large-en-2023-08-17-10-15-55-952


In [45]:
endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-endpoint"

endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.g4dn.xlarge",
            "InitialInstanceCount": 1,
            # "VolumeSizeInGB" : 400,
            # "ModelDataDownloadTimeoutInSeconds": 2400,
            "ContainerStartupHealthCheckTimeoutInSeconds": 10*60,
        },
    ],
)
endpoint_config_response

{'EndpointConfigArn': 'arn:aws:sagemaker:us-east-2:946277762357:endpoint-config/bge-large-en-2023-08-17-10-15-55-952-config',
 'ResponseMetadata': {'RequestId': 'ea0ad849-aa76-4174-873d-7df7648adbd8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'ea0ad849-aa76-4174-873d-7df7648adbd8',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '124',
   'date': 'Thu, 17 Aug 2023 10:16:02 GMT'},
  'RetryAttempts': 0}}

In [46]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

Created Endpoint: arn:aws:sagemaker:us-east-2:946277762357:endpoint/bge-large-en-2023-08-17-10-15-55-952-endpoint


In [47]:
import time

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws:sagemaker:us-east-2:946277762357:endpoint/bge-large-en-2023-08-17-10-15-55-952-endpoint
Status: InService


### 5. 设置autoscaling(optional)

In [ ]:
asg = boto3.client('application-autoscaling')

# Resource type is variant and the unique identifier is the resource ID.
resource_id=f"endpoint/{endpoint_name}/variant/variant1"

# scaling configuration
response = asg.register_scalable_target(
    ServiceNamespace='sagemaker', #
    ResourceId=resource_id,
    ScalableDimension='sagemaker:variant:DesiredInstanceCount', 
    MinCapacity=1,
    MaxCapacity=4
)

In [ ]:
response = asg.put_scaling_policy(
    PolicyName=f'Request-ScalingPolicy-{endpoint_name}',
    ServiceNamespace='sagemaker',
    ResourceId=resource_id,
    ScalableDimension='sagemaker:variant:DesiredInstanceCount',
    PolicyType='TargetTrackingScaling',
    TargetTrackingScalingPolicyConfiguration={
        'TargetValue': 10.0, # Threshold
        'PredefinedMetricSpecification': {
            'PredefinedMetricType': 'SageMakerVariantInvocationsPerInstance',
        },
        'ScaleInCooldown': 300, # duration until scale in
        'ScaleOutCooldown': 60 # duration between scale out
    }
)

### 6. 模型测试

In [48]:
def get_vector_by_sm_endpoint(questions, sm_client, endpoint_name):
    parameters = {
    }

    response_model = sm_client.invoke_endpoint(
        EndpointName=endpoint_name,
        Body=json.dumps(
            {
                "inputs": questions,
                "is_query": True,
                "instruction" :  "Represent this sentence for searching relevant passages:"
            }
        ),
        ContentType="application/json",
    )
    # 中文instruction => 为这个句子生成表示以用于检索相关文章：
    json_str = response_model['Body'].read().decode('utf8')
    json_obj = json.loads(json_str)
    embeddings = json_obj['sentence_embeddings']
    return embeddings

In [49]:
prompts1 = ["what is the default brightness setting on this device?", "how are you going"]

emb = get_vector_by_sm_endpoint(prompts1, smr_client, endpoint_name)
print(len(emb[0]))
print(emb)

1024
[[-0.3202632963657379, -0.35580018162727356, 0.22652894258499146, 0.19627542793750763, -0.6515024304389954, -0.23634003102779388, 0.006288265343755484, -0.007732868194580078, 0.14111003279685974, 0.514111340045929, 0.20397549867630005, -0.2515910267829895, 0.4812667667865753, -0.4476335644721985, -0.33535605669021606, -0.330646812915802, -0.7489869594573975, -0.3768373429775238, -0.13736592233181, 0.204591304063797, -0.13429085910320282, 0.13185717165470123, -1.2364554405212402, -0.5269641876220703, -0.5693536996841431, 0.9334635138511658, -0.13673290610313416, 0.2575847804546356, 1.2615278959274292, 1.2323713302612305, -0.4911298155784607, -0.6189811825752258, 0.515214204788208, -0.6988679766654968, -0.4431953430175781, -0.1529766470193863, 0.26056113839149475, -0.02504306472837925, -0.7690331935882568, -0.30944889783859253, 0.8968692421913147, -0.09690576791763306, 0.5038586258888245, -1.0798859596252441, -0.7485232949256897, -0.0613873153924942, -0.5474157333374023, -0.73091644

In [51]:
!aws sagemaker delete-endpoint --endpoint-name {endpoint_name}

In [52]:
!aws sagemaker delete-endpoint-config --endpoint-config-name {endpoint_config_name}

In [53]:
!aws sagemaker delete-model --model-name {model_name}